We have the data on conductor, rank, regulator and analytic order of Tate-Shafarevich group for CN curves with n upto 1,000,000 from LMFDB's 'congruent number curves' repository. We add the data of 2-Selmer ranks for all these curves. 

In [ ]:
from sage.all import *
from sage.parallel.decorate import parallel
import csv
import itertools
import time


INPUT_FILE = "CN_merged_data.csv"
OUTPUT_FILE = "cn_with_2selmer.csv"
CHUNK_SIZE = 2000

def is_squarefree(n):
    return n == n.squarefree_part()

magma = Magma()

@parallel(ncpus=15)  # Adjust ncpus to match your machine
def process_rows_parallel(chunk):
    results = []
    for row in chunk:
        try:
            n = Integer(row[0])
            a4 = -(n)**2  # Already minimal for squarefree n
            magma_code = f'''
                E := EllipticCurve([{0}, {0}, {0}, {a4}, {0}]);
                S := TwoSelmerGroup(E);
                #Generators(S);
                '''
            selmer2_rank = magma.eval(magma_code).strip()
            results.append(row[0:] + [selmer2_rank] )
        except Exception as e:
            print(f"Error processing n={row[0]}: {e}")
    return results

def stream_process_csv():
    with open(INPUT_FILE, newline='') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        data_iter = iter(reader)

        # Initialize output file with header if it doesn't exist
        if not os.path.exists(OUTPUT_FILE):
            with open(OUTPUT_FILE, 'w', newline='') as outfile:
                writer = csv.writer(outfile)
                writer.writerow(header[0:] + ['2selmer_rank'])

        while True:
            chunk = list(itertools.islice(data_iter, CHUNK_SIZE))
            if not chunk:
                break

            for _, result in process_rows_parallel([chunk]):
                with open(OUTPUT_FILE, 'a', newline='') as outfile:
                    writer = csv.writer(outfile)
                    for row in result:
                        writer.writerow(row)

# Run main processing
stream_process_csv()


In [17]:
n = 1026226
a4 = -(n)**2
E = EllipticCurve([a4,0])
print(f"{E.two_descent(second_limit=13)}")
print(f"{E.analytic_rank(algorithm='zero_sum')}")


3 points of order 2:
[-1026226:0:1], [0:0:1], [1026226:0:1]

****************************
* Using 2-isogeny number 1 *
****************************

False
Using 2-isogenous curve [0,6157356,0,1053139803076,0] (minimal model [0,0,0,-11584537833836,15130632265720596464])
-------------------------------------------------------
First step, determining 1st descent Selmer groups
-------------------------------------------------------
After first local descent, rank bound = 2
rk(S^{phi}(E'))=	2
rk(S^{phi'}(E))=	2

-------------------------------------------------------
Second step, determining 2nd descent Selmer groups
-------------------------------------------------------
After second local descent, rank bound = 2
rk(phi'(S^{2}(E)))=	2
rk(phi(S^{2}(E')))=	2
rk(S^{2}(E))=	4
rk(S^{2}(E'))=	3

****************************
* Using 2-isogeny number 2 *
****************************

Using 2-isogenous curve [0,0,0,4212559212304,0] (minimal model [0,0,0,263284950769,0])
---------------------------